# 1 - Sequence to Sequence Learning with Neural Networks

В этой статье из серии мы построим модель машинного обучения для преобразования одной последовательности в другую, используя PyTorch и torchtext. Это будет продемонстрировано на примере переводов с немецкого на английский. Однако эта модель и все последующие могут быть применены к любой проблеме, которая предполагает преобразование одной последовательности в другую, например, для задачи обобщения, то есть преобразования длинной последовательности в более короткую на том же языке, или к задаче предсказания пространственной третичной структуры белков на основе последовательности нуклеотидной цепочки мРНК ([AlphaFold](https://deepmind.com/blog/article/alphafold-a-solution-to-a-50-year-old-grand-challenge-in-biology)).

В этой статье-скрипте для понимания общих концепций моделей seq2seq мы начнем с простого примера на основе модели из статьи [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215).

## Введение

Наиболее распространенными моделями sequence-to-sequence (seq2seq) являются модели *кодера-декодера*, которые обычно используют *рекуррентную нейронную сеть* (RNN) для *кодирования* исходного (входного) предложения в один вектор. Здесьи и далее в серии мы будем называть этот единственный вектор *вектором контекста*. Мы можем думать о векторе контекста как об абстрактном представлении всего входного предложения. С точки зрения когнитивистов, этот вектор контекста — набор образов сущностей с образами взаимоотношений между ними. Затем данный вектор *декодируется* второй RNN, которая учится выводить целевое (выходное) предложение, генерируя его слово за словом, по одной ликсеме за раз.

![](assets/seq2seq1.png)

На изображении выше показан пример перевода. Предложение ввода/источника, «guten morgen», проходит через слой эмбеддинга (желтый, служит для сопоставления элементов речи (слова, предложения,...) числовому вектору), а затем вводится в кодировщик (зеленый). Всегда добавляются токены *начало последовательности* (`<sos>` — *start of sequence*) и *конец последовательности* (`<eos>` — *end of sequence*) в начало и конец предложения соответственно. На каждом временном шаге на вход в RNN кодировщика подаётся эмбеддинг-версия текущего слова, $ e (x_t)$, которая порождена слоем эмбеддинга $ e $, так и скрытое состояние из предыдущего временного шага, $ h_{t-1} $. На выход RNN кодировщика подаёт новое скрытое состояние $ h_t $. Здесь мы можем думать о скрытом состоянии как о векторном представлении предложения. RNN кодировщика может быть представлена как функция (EncoderRNN) от переменных $ e (x_t) $ и $ h_ {t-1} $:

$$h_t = \text{EncoderRNN}(e(x_t), h_{t-1})$$

Здесь и далее используя аббревиатуру RNN, подразумеваем под ней сеть любой рекуррентной архитектуры: например, *LSTM* (Long Short-Term Memory) или *GRU* (Gated Recurrent Unit).

Здесь мы имеем $X = \{x_1, x_2, ..., x_T \}$, где $ x_1 = \text{<sos>}, x_{2} = \text{guten}$ и т.д. Начальное скрытое состояние, $h_0$, обычно либо инициализируется нулями, либо является параметром обучения.

После того как последнее слово, $ x_T $, было передано в RNN через слой эмбеддинга, мы используем конечное скрытое состояние, $ h_T $, как вектор контекста, то есть $ z = h_T $. Это векторное представление всего исходного предложения.

Теперь у нас есть вектор контекста $ z $, который мы можем начать декодировать, чтобы получить выходное/целевое предложение «доброе утро». Как и в предыдущем случае, мы добавляем токены начала и конца последовательности к целевому предложению. На каждом временном шаге входом в RNN декодера (синий) является эмбеддинг текущего слова, $ d (y_t) $, а также скрытое состояние из предыдущего временного шага, $ s_ { t-1} $, где начальное скрытое состояние декодера, $ s_0 $, является вектором контекста, $ s_0 = z $. Т.е. начальное скрытое состояние декодера является окончательным скрытым состоянием кодера. Таким образом, аналогично кодеру, мы можем представить декодер как функцию от переменных $ d (y_t) $ и s_ { t-1}:

$$s_t = \text{DecoderRNN}(d(y_t), s_{t-1})$$

Хотя слой эмбеддинга входа/источника $e$ и слой эмбеддинга выхода $d$ оба показаны жёлтым цветом на приведённом рисунке, они представляют собой два разных слоя эмбеддинга со своими собственными параметрами.

В декодере нам нужно перейти от скрытого состояния к фактическому слову, поэтому на каждом временном шаге мы используем $ s_t $ для распознавания (передавая эту величину через слой `Linear`, показанный фиолетовым цветом) того, что мы считаем следующим словом в последовательности $ \ hat {y} _t $.

$$\hat{y}_t = f(s_t)$$

Слова в декодере всегда генерируются одно за другим, по одному за один временной шаг. Мы всегда используем `<sos>` в качестве первого слова $ y_1 $ для ввода в декодер, но для последующих слов для ввода, $ y_ {t> 1} $, мы иногда будем использовать фактическое, истинное/целевое следующее слово в последовательности $ y_t $, и иногда будем брать слово, предложенное нашим декодером, $ \ hat {y} _ {t-1} $. Такой метод обучения называется *обучением с принуждением*, подробнее об этом [здесь](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/).

При обучении/тестировании нашей модели мы всегда знаем, сколько слов в нашем целевом предложении, поэтому мы перестаем генерировать слова, как только набираем это количество. Во время логического вывода обычно генерируют слова до тех пор, пока модель не выдаст токен окончания последовательности <eos> или после того, как будет сгенерировано определенное количество слов.

Когда у нас есть предсказанное предложение, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$, мы сравниваем его с нашим фактическим целевым предложение, $Y = \{ y_1, y_2, ..., y_T \}$, для расчета потерь/ошибок. Затем мы используем эти потери для обновления всех параметров обучаемой модели.

## Подготовка данных

Мы будем описывать модель в PyTorch и использовать torchtext для выполнения всей необходимой предварительной обработки. Кроме того, мы будем использовать spaCy для помощи в токенизации данных.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

Мы установим случайные начальные числа для получения детерминированных результатов.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Далее мы создадим токенизаторы. Токенизатор используется для превращения строки, содержащей предложение, в список отдельных токенов, составляющих эту строку, например "доброе утро!" становится ["доброе", "утро", "!"]. С этого момента мы начнем говорить о предложениях, представляющих собой последовательность токенов, вместо того, чтобы говорить, что они являются последовательностью слов. Какая разница? Что ж, «добро» и «утро» - это и слова, и знаки, но «!» это знак, а не слово.

В spaCy есть модель для каждого языка (de_core_news_sm для немецкого и en_core_web_sm для английского), которую необходимо загрузить, чтобы мы могли получить доступ к токенизатору каждой модели.

**Примечание**: сначала необходимо загрузить модели на диск, используя следующую команду в командной строке:

```
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```

Мы загружаем модели в память следующим образом:

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

Далее мы создаем токенизирующие функции. Они могут быть переданы в torchtext и будут принимать предложение в виде строки, а возвращать предложение в виде списка токенов.

В статье, на основе которой мы реализуем данную модель, авторы считают полезным изменить порядок ввода токенов, который, по их мнению, «вводит множество краткосрочных зависимостей в данные, значительно упрощая задачу оптимизации». Мы копируем результат, переворачивая немецкое предложение после того, как оно было преобразовано в список токенов.

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Класс `Field` в torchtext указывает как данные должны быть обработаны. Все возможные аргументы детально описаны [здесь](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L61).

Мы устанавливаем правильную функцию токенизации в качестве аргумента `tokenize` для каждого языка, причем немецкий является полем` SRC` (источник), а английский — полем `TRG` (цель). Экземпляры класса `Field` содержат токены «начало последовательности» и «конец последовательности» как аргументы «init_token» и «eos_token». Все слова преобразуются к нижнему регистру.

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Далее скачиваем и загружаем данные для обучения, валидации во время обучения и тестирования.

Набор данных, который мы будем использовать, - это [набор данных Multi30k](https://github.com/multi30k/dataset). Это набор данных из ~ 30 000 параллельных предложений на английском, немецком и французском языках, каждое из которых содержит порядка 12-ти слов в предложении.

`exts` указывает, какие языки использовать в качестве источника и цели (источник идет первым), а `fields` указывает, какое поле использовать для источника и цели.

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Мы можем проверить, что мы загрузили нужное количество примеров:

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


Мы также можем распечатать пример, убедившись, что исходное предложение перевернуто:

In [8]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


Точка стоит в начале предложения на немецком языке (src), поэтому похоже, что предложение было правильно перевернуто.

Затем мы создадим *словарь* для исходного и целевого языков. Словарь используется для связывания каждого уникального токена с индексом (целым числом). Словари исходного и целевого языков различаются.

Используя аргумент `min_freq`, мы разрешаем только токенам, которые появляются как минимум 2 раза, появляться в нашем словаре. Токены, которые появляются только один раз, конвертируются в токен `<unk>` (неизвестно).

Важно отметить, что наш словарь должен строиться только на основе обучающего набора, а не набора для проверки/тестирования. Это предотвращает «утечку информации» в нашу модель, давая нам искусственно завышенные оценки валидации/тестирования.

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


Последний шаг подготовки данных — создание итераторов. С их помощью можно итерационно возвращать пакеты данных, которые будут иметь атрибут `src` (тензоры PyTorch, содержащие набор оцифрованных исходных предложений) и атрибут `trg` (тензоры PyTorch, содержащие набор оцифрованных целевых предложений). Оцифрованные предложения — это просто причудливый способ сказать, что они были преобразованы из последовательности читаемых токенов в последовательность соответствующих индексов с использованием словаря.

Нам также нужно определить `torch.device`. Эта величина используется, чтобы указать torchText, куда помещать тензоры для вычислений: на GPU или нет? Мы используем функцию `torch.cuda.is_available()`, которая вернет `True`, если на нашем компьютере обнаружен графический процессор. Мы передаем это поле итератору.

Когда мы получаем батч примеров с использованием итератора, нам нужно убедиться, что все исходные предложения дополнены до одинаковой длины, как и целевые предложения. К счастью, итераторы torchText справятся с этим за нас!

Мы используем `BucketIterator` вместо стандартного `Iterator`, поскольку он создает пакеты таким образом, чтобы минимизировать количество отступов как в исходном, так и в целевом предложении.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Построение модели Seq2Seq

Мы будем строить нашу модель из трех частей. Кодер, декодер и модель seq2seq, которая инкапсулирует кодер и декодер, и обеспечивает способ взаимодействия с каждым из них.

### Кодер

Во-первых, кодер — это двухслойная LSTM сеть. В статье, на которую мы опираемся при реализации, используется 4-слойная LSTM сеть, но в целях экономии времени мы сократили количество слоёв до 2-х. Многослойную RNN легко расширить с 2 до 4 уровней.

Во многослойной RNN входное предложение $X$ после эмбеддинга передаётся в первый (нижний) слой RNN и порождает скрытые состояния $H=\{h_1, h_2, ..., h_T\}$. Выходные данные этого слоя используются в качестве входных данных для RNN на уровне выше. Таким образом, представляя каждый слой надстрочным индексом, скрытые состояния в первом слое задаются следующим образом:

$$h_t^1 = \text{EncoderRNN}^1(e(x_t), h_{t-1}^1)$$

Скрытые состояния во втором слое задаются следующим образом:

$$h_t^2 = \text{EncoderRNN}^2(h_t^1, h_{t-1}^2)$$

Использование многослойной RNN предполагает, что нам понадобится исходное скрытое состояние в качестве входных данных для каждого слоя $h_0^l$, и сеть будет предоставлять вектор контекста для каждого слоя $z^l$.

Не вдаваясь в подробности о LSTM сети (см. [здесь](https://colah.github.io/posts/2015-08-Understanding-LSTMs/), чтобы узнать о них больше), все, что нам нужно знать, это то, что эти сети являются RNN, которые вместо того, чтобы просто принимать скрытое состояние и возвращать новое скрытое состояние для каждого временного шага, принимают и возвращают *состояние ячейки* $ c_t $, для каждого временного шага.

$$\begin{align*}
h_t &= \text{RNN}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})
\end{align*}$$

Мы можем принять $c_t$ как другой тип скрытого состояния. Подобно $h_0^l$, величина $c_0^l$ будет инициализирована тензором, заполненным нулями. Кроме того, наш вектор контекста теперь будет образован конечным скрытым состоянием и конечным состоянием ячейки, то есть $z^l = (h_T^l, c_T^l)$.

Распространяя наши многослойные уравнения на LSTM, мы получаем:

$$\begin{align*}
(h_t^1, c_t^1) &= \text{EncoderLSTM}^1(e(x_t), (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2) &= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Обратите внимание на то, что наше скрытое состояние из первого слоя передается в качестве входных данных во второй уровень, а не состояние ячейки.

Итак, наш кодировщик выглядит примерно так:

![](assets/seq2seq2.png)

Мы реализуем это в коде, создав модуль `Encoder`, который требует, чтобы мы унаследовали это класс от` torch.nn.Module` и использовали `super () .__ init __ ()` как некоторый шаблонный код. Кодировщик принимает следующие аргументы:
- `input_dim` - размер / размерность one-hot векторов, которые будут вводиться в кодировщик. Они равны размеру входного (исходного) размера словаря.
- `emb_dim` - размерность слоя эмбеддинга. Этот слой преобразует one-hot векторы в плотные векторы с размерами `emb_dim`.
- `hid_dim` - размерность скрытого состояния и состояния ячейки.
- `n_layers` - количество слоев в RNN.
- `dropout` - это количество использованных отсевов. Это параметр регуляризации для предотвращения переобучения. Ознакомьтесь с [этим здесь](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) для получения дополнительных сведений о прекращении обучения.

Мы не собираемся подробно обсуждать слой эмбеддинга в этих заметках. Все, что нам нужно знать, это то, что есть шаг до того, как слова - технически индексы слов - передаются в RNN, где слова преобразуются в векторы. Чтобы узнать больше о встраивании слов, посмотрите эти статьи: [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/).

Слой эмбеддинга создаётся с помощью `nn.Embedding`, LSTM с `nn.LSTM` и слой дропаута с `nn.Dropout`. Дополнительные сведения об этом см. [документации](https://pytorch.org/docs/stable/nn.html).

Следует отметить, что аргумент `dropout` для LSTM заключается в том, сколько отсева необходимо применять между уровнями многослойной RNN, то есть между скрытыми состояниями, выводимыми из уровня $ l $, и теми же скрытыми состояниями, используемыми для ввод слоя $ l + 1 $.

В методе `forward` мы передаем исходное предложение $ X $, которое преобразуется в плотные векторы с помощью слоя `embedding`, а затем применяется дропаут. Эти величины через эмбеддинг затем передаются в RNN. Когда мы передаем всю последовательность в RNN, она автоматически выполняет для нас повторяющийся расчет скрытых состояний по всей последовательности! Обратите внимание, что мы не передаем в RNN начальное скрытое состояние или состояние ячейки. Это связано с тем, что, как указано в [документации] (https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), если в RNN не передается скрытое состояние / состояние ячейки, оно автоматически создает начальное скрытое состояние / состояние ячейки как тензор всех нулей.

RNN возвращает: `выходные` (скрытые состояния верхнего уровня для каждого временного шага), `скрытые` (окончательное скрытое состояние для каждого слоя, $h_T$, наложенное друг на друга) и `ячейка`»` (конечное состояние ячейки для каждого слоя, $c_T$, наложенных друг на друга).

Поскольку нам нужны только окончательные скрытой состояния и состояние ячейки (для создания нашего вектора контекста), `forward` только возвращает `hidden` и `cell`.

Размеры каждого из тензоров оставлены в виде комментариев в коде. В этой реализации `n_directions` всегда будет 1, однако обратите внимание, что двунаправленные RNN (описанные в учебнике 3) будут иметь` n_directions` как 2.

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Decoder

Next, we'll build our decoder, which will also be a 2-layer (4 in the paper) LSTM.

![](assets/seq2seq3.png)

The `Decoder` class does a single step of decoding, i.e. it ouputs single token per time-step. The first layer will receive a hidden and cell state from the previous time-step, $(s_{t-1}^1, c_{t-1}^1)$, and feeds it through the LSTM with the current embedded token, $y_t$, to produce a new hidden and cell state, $(s_t^1, c_t^1)$. The subsequent layers will use the hidden state from the layer below, $s_t^{l-1}$, and the previous hidden and cell states from their layer, $(s_{t-1}^l, c_{t-1}^l)$. This provides equations very similar to those in the encoder.

$$\begin{align*}
(s_t^1, c_t^1) = \text{DecoderLSTM}^1(d(y_t), (s_{t-1}^1, c_{t-1}^1))\\
(s_t^2, c_t^2) = \text{DecoderLSTM}^2(s_t^1, (s_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Remember that the initial hidden and cell states to our decoder are our context vectors, which are the final hidden and cell states of our encoder from the same layer, i.e. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.

We then pass the hidden state from the top layer of the RNN, $s_t^L$, through a linear layer, $f$, to make a prediction of what the next token in the target (output) sequence should be, $\hat{y}_{t+1}$. 

$$\hat{y}_{t+1} = f(s_t^L)$$

The arguments and initialization are similar to the `Encoder` class, except we now have an `output_dim` which is the size of the vocabulary for the output/target. There is also the addition of the `Linear` layer, used to make the predictions from the top layer hidden state.

Within the `forward` method, we accept a batch of input tokens, previous hidden states and previous cell states. As we are only decoding one token at a time, the input tokens will always have a sequence length of 1. We `unsqueeze` the input tokens to add a sentence length dimension of 1. Then, similar to the encoder, we pass through an embedding layer and apply dropout. This batch of embedded tokens is then passed into the RNN with the previous hidden and cell states. This produces an `output` (hidden state from the top layer of the RNN), a new `hidden` state (one for each layer, stacked on top of each other) and a new `cell` state (also one per layer, stacked on top of each other). We then pass the `output` (after getting rid of the sentence length dimension) through the linear layer to receive our `prediction`. We then return the `prediction`, the new `hidden` state and the new `cell` state.

**Note**: as we always have a sequence length of 1, we could use `nn.LSTMCell`, instead of `nn.LSTM`, as it is designed to handle a batch of inputs that aren't necessarily in a sequence. `nn.LSTMCell` is just a single cell and `nn.LSTM` is a wrapper around potentially multiple cells. Using the `nn.LSTMCell` in this case would mean we don't have to `unsqueeze` to add a fake sequence length dimension, but we would need one `nn.LSTMCell` per layer in the decoder and to ensure each `nn.LSTMCell` receives the correct initial hidden state from the encoder. All of this makes the code less concise - hence the decision to stick with the regular `nn.LSTM`.

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

### Seq2Seq

For the final part of the implemenetation, we'll implement the seq2seq model. This will handle: 
- receiving the input/source sentence
- using the encoder to produce the context vectors 
- using the decoder to produce the predicted output/target sentence

Our full model will look like this:

![](assets/seq2seq4.png)

The `Seq2Seq` model takes in an `Encoder`, `Decoder`, and a `device` (used to place tensors on the GPU, if it exists).

For this implementation, we have to ensure that the number of layers and the hidden (and cell) dimensions are equal in the `Encoder` and `Decoder`. This is not always the case, we do not necessarily need the same number of layers or the same hidden dimension sizes in a sequence-to-sequence model. However, if we did something like having a different number of layers then we would need to make decisions about how this is handled. For example, if our encoder has 2 layers and our decoder only has 1, how is this handled? Do we average the two context vectors output by the decoder? Do we pass both through a linear layer? Do we only use the context vector from the highest layer? Etc.

Our `forward` method takes the source sentence, target sentence and a teacher-forcing ratio. The teacher forcing ratio is used when training our model. When decoding, at each time-step we will predict what the next token in the target sequence will be from the previous tokens decoded, $\hat{y}_{t+1}=f(s_t^L)$. With probability equal to the teaching forcing ratio (`teacher_forcing_ratio`) we will use the actual ground-truth next token in the sequence as the input to the decoder during the next time-step. However, with probability `1 - teacher_forcing_ratio`, we will use the token that the model predicted as the next input to the model, even if it doesn't match the actual next token in the sequence.  

The first thing we do in the `forward` method is to create an `outputs` tensor that will store all of our predictions, $\hat{Y}$.

We then feed the input/source sentence, `src`, into the encoder and receive out final hidden and cell states.

The first input to the decoder is the start of sequence (`<sos>`) token. As our `trg` tensor already has the `<sos>` token appended (all the way back when we defined the `init_token` in our `TRG` field) we get our $y_1$ by slicing into it. We know how long our target sentences should be (`max_len`), so we loop that many times. The last token input into the decoder is the one **before** the `<eos>` token - the `<eos>` token is never input into the decoder. 

During each iteration of the loop, we:
- pass the input, previous hidden and previous cell states ($y_t, s_{t-1}, c_{t-1}$) into the decoder
- receive a prediction, next hidden state and next cell state ($\hat{y}_{t+1}, s_{t}, c_{t}$) from the decoder
- place our prediction, $\hat{y}_{t+1}$/`output` in our tensor of predictions, $\hat{Y}$/`outputs`
- decide if we are going to "teacher force" or not
    - if we do, the next `input` is the ground-truth next token in the sequence, $y_{t+1}$/`trg[t]`
    - if we don't, the next `input` is the predicted next token in the sequence, $\hat{y}_{t+1}$/`top1`, which we get by doing an `argmax` over the output tensor
    
Once we've made all of our predictions, we return our tensor full of predictions, $\hat{Y}$/`outputs`.

**Note**: our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Later on when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

# Training the Seq2Seq Model

Now we have our model implemented, we can begin training it. 

First, we'll initialize our model. As mentioned before, the input and output dimensions are defined by the size of the vocabulary. The embedding dimesions and dropout for the encoder and decoder can be different, but the number of layers and the size of the hidden/cell states must be the same. 

We then define the encoder, decoder and then our Seq2Seq model, which we place on the `device`.

In [16]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

Next up is initializing the weights of our model. In the paper they state they initialize all weights from a uniform distribution between -0.08 and +0.08, i.e. $\mathcal{U}(-0.08, 0.08)$.

We initialize weights in PyTorch by creating a function which we `apply` to our model. When using `apply`, the `init_weights` function will be called on every module and sub-module within our model. For each module we loop through all of the parameters and sample them from a uniform distribution with `nn.init.uniform_`.

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We also define a function that will calculate the number of trainable parameters in the model.

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.

In [19]:
optimizer = optim.Adam(model.parameters())

Next, we define our loss function. The `CrossEntropyLoss` function calculates both the log softmax as well as the negative log-likelihood of our predictions. 

Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [20]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Next, we'll define our training loop. 

First, we'll set the model into "training mode" with `model.train()`. This will turn on dropout (and batch normalization, which we aren't using) and then iterate through our data iterator.

As stated before, our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Here, when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we slice off the first column of the output and target tensors as mentioned above
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [21]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used).

We use the `with torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up. 

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [22]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Next, we'll create a function that we'll use to tell us how long an epoch takes.

In [23]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We can finally start training our model!

At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called `state_dict` in PyTorch). Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss. 

We'll be printing out both the loss and the perplexity at each epoch. It is easier to see a change in perplexity than a change in loss as the numbers are much bigger.

In [24]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Time: 0m 26s
	Train Loss: 5.052 | Train PPL: 156.386
	 Val. Loss: 4.916 |  Val. PPL: 136.446
Epoch: 02 | Time: 0m 26s
	Train Loss: 4.483 | Train PPL:  88.521
	 Val. Loss: 4.789 |  Val. PPL: 120.154
Epoch: 03 | Time: 0m 25s
	Train Loss: 4.195 | Train PPL:  66.363
	 Val. Loss: 4.552 |  Val. PPL:  94.854
Epoch: 04 | Time: 0m 25s
	Train Loss: 3.963 | Train PPL:  52.625
	 Val. Loss: 4.485 |  Val. PPL:  88.672
Epoch: 05 | Time: 0m 25s
	Train Loss: 3.783 | Train PPL:  43.955
	 Val. Loss: 4.375 |  Val. PPL:  79.466
Epoch: 06 | Time: 0m 25s
	Train Loss: 3.636 | Train PPL:  37.957
	 Val. Loss: 4.234 |  Val. PPL:  69.011
Epoch: 07 | Time: 0m 26s
	Train Loss: 3.506 | Train PPL:  33.329
	 Val. Loss: 4.077 |  Val. PPL:  58.948
Epoch: 08 | Time: 0m 27s
	Train Loss: 3.370 | Train PPL:  29.090
	 Val. Loss: 4.018 |  Val. PPL:  55.581
Epoch: 09 | Time: 0m 26s
	Train Loss: 3.241 | Train PPL:  25.569
	 Val. Loss: 3.934 |  Val. PPL:  51.113
Epoch: 10 | Time: 0m 26s
	Train Loss: 3.157 | Train PPL

We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [25]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.951 | Test PPL:  52.001 |


In the following notebook we'll implement a model that achieves improved test perplexity, but only uses a single layer in the encoder and the decoder.